In [7]:
%pip install -qU langchain-pinecone pinecone-notebooks langchain-openai langchain-community tiktoken

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install "pinecone[grpc]" -qU

Note: you may need to restart the kernel to use updated packages.


In [24]:
PINECONE_API_KEY = ""
PINECONE_ENVIRONMENT=""
PINECONE_INDEX=""

OPENAI_API_KEY = ""
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [29]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [31]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="text")


In [32]:
USER_1 = "Alex"
USER_2 = "Lucy"

In [33]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/multi_user/whitepaper-v3.pdf")
documents = loader.load_and_split()
vectorstore.add_documents(documents, namespace=USER_2)

['da9d57c6-360c-48b8-88c9-3e91a31f6267',
 '785bf668-403a-4302-909b-9fe871641349',
 '50c02a1d-b88f-411f-8460-87f1397eaf1a',
 '1ca7c3da-0723-4074-ae1c-82e3e687c5e8',
 '0fd308d2-952c-41ac-ab21-885000c5caab',
 'fd8c3a21-df03-4a1f-8008-6f505bbbaa79',
 'd237e7e3-c81d-4446-ae43-bdf31da7c9c7',
 'a06b44fa-f8dc-474c-9f97-f237c1c51633',
 '40cf4ae5-e92e-4d6c-bcd7-d6119c428aa6',
 '0ef37895-8e4c-4c61-9b1d-62830050edc6',
 '185483c9-c0b8-4bb7-b0a3-4d00ce47ea43',
 'b2f39b67-6119-45da-9e49-1f7234077d11',
 'ef193fe8-907a-48ab-91d5-a46f7acba700',
 'f0cda25d-dd35-4663-af0a-949a2f0a5eab']

In [34]:
loader = PyPDFLoader("data/multi_user/SM-EHR-White-Papers_v12.pdf")
documents = loader.load_and_split()
vectorstore.add_documents(documents, namespace=USER_1)

['53482f66-ff58-4604-9647-6b72cf5a02da',
 'e178621f-7a28-482e-bab8-acb94e9fc7b3',
 'ad64060b-2a88-47f0-9c4e-fdd8b11597eb',
 '16e4af27-6a9d-4bb3-b4c6-27acab5cb04e',
 '42078b68-2fd1-47ca-8a20-cdda52729327',
 '1325c252-b225-4394-b4a8-a187eb7ea413',
 '1a46f554-6079-4473-ba95-5478f5a973bb',
 'f27690ff-4512-4668-96df-127e961917e8',
 'fbea49b9-03e0-4f61-a3b2-ff6d443a9008',
 '2b17f63f-d188-4b7d-aaa0-b2fdec1b7546',
 'f578d006-17ef-42e5-8a60-ac08a8f1ba2b',
 '4668bef8-7bc4-4893-99e1-018024f48b45',
 '08b16332-a740-4c74-8351-396c51ecc7a3',
 'ef2c9a38-cd35-4ef1-a1fc-aa641a80c443',
 'a1ec20c8-1e2d-4e6c-ba84-9ee22e22935e',
 'b37a60c9-0d8e-4dbd-8091-3e9578c7653e',
 '87e25ce6-9bbf-4f38-b67a-17651f9a26c6',
 'f30a09ad-66c2-4e21-8e56-46c21c8230fe',
 'f409571b-36e2-475e-b9e9-1a9202202421',
 '76549251-0372-408a-9100-f80a6cf6306a',
 '5d9f5116-b01f-4c86-a2f7-b77f2592fda4']

In [35]:
question = "What is uniswap?"

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [36]:

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"namespace": USER_1, "score_threshold": 0.9})
relevant_documents = retriever.invoke(question)
relevant_documents

No relevant docs were retrieved using the relevance score threshold 0.9


[]

In [37]:
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"namespace": USER_2, "score_threshold": 0.9})
relevant_documents = retriever.invoke(question)
relevant_documents

[Document(metadata={'page': 0.0, 'source': 'data/multi_user/whitepaper-v3.pdf'}, page_content='Uniswap v3 Core\nMarch 2021\nHayden Adams\nhayden@uniswap.orgNoah Zinsmeister\nnoah@uniswap.orgMoody Salem\nmoody@uniswap.org\nRiver Keefer\nriver@uniswap.orgDan Robinson\ndan@paradigm.xyz\nABSTRACT\nUniswap v3 is a noncustodial automated market maker imple-\nmented for the Ethereum Virtual Machine. In comparison to earlier\nversions of the protocol, Uniswap v3 provides increased capital\nefficiency and fine-tuned control to liquidity providers, improves\nthe accuracy and convenience of the price oracle, and has a more\nflexible fee structure.\n1 INTRODUCTION\nAutomated market makers (AMMs) are agents that pool liquidity\nand make it available to traders according to an algorithm [ 5]. Con-\nstant function market makers (CFMMs), a broad class of AMMs of\nwhich Uniswap is a member, have seen widespread use in the con-\ntext of decentralized finance, where they are typically implemented\nas sma

In [43]:
def ask_question_by_user(user, question):
    retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"namespace": user, "score_threshold": 0.9})
    template = """Answer the question based only on the following context which is wrapped by tags [CONTEXT][/context]:
    [CONTEXT]
    {context}
    [/CONTEXT]
    If you don't know the answer, just say that you don't know. Don't make up an answer.
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    model = ChatOpenAI()

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    response = retrieval_chain.invoke(question)
    print(response)

In [46]:
query = "What would a health caresystem with a fluid movement of information and state-of-the-art technology look like?"

In [47]:
ask_question_by_user(USER_1, query)


The Vision for 2028 describes a health care system with a fluid movement of information and state-of-the-art technology as one where physicians, nurses, and other health care practitioners can focus on patient care without having to worry about health records. Information would flow seamlessly between all necessary parties, including insurance companies, hospitals, physicians, and patients. The electronic health record would be automatically populated with relevant information, and an automated physician's assistant would assist in recording and synthesizing medical information. The goal is to bring Precision Health into practice and keep people healthy by providing real-time diagnostic support and personalized treatment options.


In [48]:
ask_question_by_user(USER_2, query)

No relevant docs were retrieved using the relevance score threshold 0.9


I'm sorry, I don't know.
